In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
# Requests
import requests
from time import sleep
from tqdm import tqdm

# Para convertir de string a diccionario
import ast
import os
import json

import sys
sys.path.append("../")
import src.soporte_spotify as api

In [2]:
sp = api.load_credentials()
sp.current_user()

{'country': 'ES',
 'display_name': 'MrApple',
 'email': 'ruiperezgonzalo4@gmail.com',
 'explicit_content': {'filter_enabled': False, 'filter_locked': False},
 'external_urls': {'spotify': 'https://open.spotify.com/user/gonzaloruiperez'},
 'followers': {'href': None, 'total': 0},
 'href': 'https://api.spotify.com/v1/users/gonzaloruiperez',
 'id': 'gonzaloruiperez',
 'images': [],
 'product': 'premium',
 'type': 'user',
 'uri': 'spotify:user:gonzaloruiperez'}

# Verificar si nos han bloqueado
- Tenemos un limite diario y si no spotify devuelve 429, algo que con spotipy no se ve, asi que usamos requests

In [3]:
token = api.request_token()
playlist_id = "1IMxg7e2dwhawHaoBfWWIH" 
url = f"https://api.spotify.com/v1/playlists/{playlist_id}"

headers = {
    "Authorization": f"Bearer {token}"
}

response = requests.get(url, headers=headers)


print(f"{response.status_code}, Detalles: {response.text}")
print(response.headers)


Token obtenido con éxito
429, Detalles: Too many requests
{'cache-control': 'private, max-age=0', 'retry-after': '80730', 'access-control-allow-origin': '*', 'access-control-allow-headers': 'Accept, App-Platform, Authorization, Content-Type, Origin, Retry-After, Spotify-App-Version, X-Cloud-Trace-Context, client-token, content-access-token', 'access-control-allow-methods': 'GET, POST, OPTIONS, PUT, DELETE, PATCH', 'access-control-allow-credentials': 'true', 'access-control-max-age': '604800', 'content-encoding': 'gzip', 'strict-transport-security': 'max-age=31536000', 'x-content-type-options': 'nosniff', 'date': 'Fri, 10 Jan 2025 11:49:13 GMT', 'server': 'envoy', 'Via': 'HTTP/2 edgeproxy, 1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000', 'Transfer-Encoding': 'chunked'}


In [4]:
zara = pd.read_csv('../datos/01 Spotify/01_followers_zara.csv',index_col=0)
primark = pd.read_csv("../datos/01 Spotify/02_followers_primark.csv",index_col=0)
nike = pd.read_csv("../datos/01 Spotify/03_followers_nike.csv",index_col=0)
adidas = pd.read_csv("../datos/01 Spotify/04_followers_adidas.csv",index_col=0)
hm = pd.read_csv("../datos/01 Spotify/05_followers_hm.csv",index_col=0)
pullbear = pd.read_csv("../datos/01 Spotify/06_followers_pull_and_bear.csv",index_col=0)
bershka = pd.read_csv("../datos/01 Spotify/07_followers_bershka.csv",index_col=0)
stradivarius = pd.read_csv("../datos/01 Spotify/08_followers_stradivarius.csv",index_col=0)
mango = pd.read_csv("../datos/01 Spotify/09_followers_mango.csv",index_col=0)
desigual = pd.read_csv("../datos/01 Spotify/10_followers_desigual.csv",index_col=0)

# Zara
### Obtener ID canciones
- Por cada id de playlist, sacaremos los tracks con su respectivo id de canción


In [5]:
zara.sample()

,brand,username,user_id,playlists
130,Zara,batel,312qhxx6buvxvyrg2sc4p6zh7ug4,"{""ג'יפסי"": '0WeOFVkewufVfnpv4LoEft', 'אברהם טל..."


### Convertir columna playlists a diccionario
- utilizamos la librería ast para obtener el diccionario

In [6]:
zara["playlists"] = zara["playlists"].apply(ast.literal_eval)
type(zara["playlists"][0])

dict

### Sacamos a una lista todos los ids de playlists y otra con máximo 10
- Limitamos a 10 playlists para que usuarios con más no desbalanceen al resto!

In [7]:
playlist_ids = []
for ids in zara["playlists"]:
    playlist_ids.append(list(ids.values()))

zara["playlist_ids"] = playlist_ids
zara["playlist_ids_limited"] = zara["playlist_ids"].apply(lambda x: x[:10] if isinstance(x,list) else x)

In [8]:
output_file = "../datos/01 Spotify/00_GuardadoZara.csv"

# Cargar progreso si existe un archivo previo
if os.path.exists(output_file):
    zara = pd.read_csv(output_file)

zara[zara["artistas"].notna()]

,brand,username,user_id,playlists,playlist_ids,playlist_ids_limited,artistas
0,Zara,! ...,31vbe3hoptpfuzrvfqxalxi5tepu,{'Best Songs Ever': '1IMxg7e2dwhawHaoBfWWIH'},['1IMxg7e2dwhawHaoBfWWIH'],['1IMxg7e2dwhawHaoBfWWIH'],"{""3qm84nBOXUEQ2vnTfUTTFC"": ""Guns N' Roses"", ""6..."
1,Zara,$Slab,31gevswudwftwxxcbfszt5o2u5ta,{'fylabstoken.netlify.app': '4YWWifgQd3JLFEFxC...,['4YWWifgQd3JLFEFxCW6x3I'],['4YWWifgQd3JLFEFxCW6x3I'],{}
2,Zara,-M's- Café & Bar,lpet16qfnsa9zn97li98q8c50,"{'Bach, J.S.: Organ Works': '0odoqvp5UQ4g80ODi...","['0odoqvp5UQ4g80ODiidFKm', '3NhcYwnSqua8GvXr6C...","['0odoqvp5UQ4g80ODiidFKm', '3NhcYwnSqua8GvXr6C...","{""5aIqB5nVVvmFsvSdExz408"": ""Johann Sebastian B..."
3,Zara,. BR STORE,22d35vgyrjr3ywbwpnolontry,{'Meu Verão Brasil ✌🏼': '00hEjNe3OBE1C9ooiR2UI...,"['00hEjNe3OBE1C9ooiR2UIV', '5alYUj2HYhBmLvreNG...","['00hEjNe3OBE1C9ooiR2UIV', '5alYUj2HYhBmLvreNG...","{""37W5idqXpnDRxbgtXBW5nN"": ""Clara Mendes"", ""3y..."
4,Zara,11120410121,11120410121,"{'Fiestas Horcajo': '1s8Uk2EJji7AxBR4qWGH1g', ...","['1s8Uk2EJji7AxBR4qWGH1g', '2ddUcOB22VC5yR2Pp7...","['1s8Uk2EJji7AxBR4qWGH1g', '2ddUcOB22VC5yR2Pp7...","{""2fiiGUBgPsIKDCE0bTthnl"": ""Ska-P"", ""6J6yx1t3n..."
...,...,...,...,...,...,...,...
262,Zara,Enes,o85kfbeziici4kkbtktmqnfsx,{'Meine Playlist Nr. 1': '6B1YFXrUMZl9lSxFco5q...,['6B1YFXrUMZl9lSxFco5qtA'],['6B1YFXrUMZl9lSxFco5qtA'],{}
263,Zara,ENGEL,424syj2h6jcy66qge77bzwuoe,{'OT 23/24 BEST BY ENGEL': '2wOY3pE2nXoGSY7vHV...,"['2wOY3pE2nXoGSY7vHVoQyJ', '4fg0AsJmaehnyLEGQQ...","['2wOY3pE2nXoGSY7vHVoQyJ', '4fg0AsJmaehnyLEGQQ...","{""0HgzUc7N5MqMv2ZwQFv9H0"": ""MAYO"", ""0aA3vWOE6E..."
264,Zara,Enio Roboredo Sanches,enio.sanches,"{'MTF': '4AzyUSLwYlrI0odWStunwR', 'To download...","['4AzyUSLwYlrI0odWStunwR', '2wpcqOYdIFCrQ3WYOx...","['4AzyUSLwYlrI0odWStunwR', '2wpcqOYdIFCrQ3WYOx...","{""3r5vRW2MU6y539p7NRcTCt"": ""88DS"", ""17TFSnqADE..."
265,Zara,eru,1285025015,{'benadryl submarine vibes': '6vWa2hywiHokpsG2...,"['6vWa2hywiHokpsG24jKqBm', '2bbghy0KLSlsWIglbP...","['6vWa2hywiHokpsG24jKqBm', '2bbghy0KLSlsWIglbP...","{""2gPLG9mGaU67woC5L0luYA"": ""Lil Ugly Mane"", ""7..."


### Obtener los artistas de cada usuario

In [ ]:
lista_artistas = []
output_file = "../datos/01 Spotify/00_GuardadoZara.csv"

# Cargar progreso si existe un archivo previo
if os.path.exists(output_file):
    zara = pd.read_csv(output_file)
    lista_artistas = zara["artistas"].tolist()
else:
    zara["artistas"] = None

# Procesar playlists limitadas
for idx, id_playlist in enumerate(tqdm(zara["playlist_ids_limited"], desc="Obteniendo artistas")):
    if pd.notna(zara.loc[idx, "artistas"]):
        continue
    print(idx)
    # Solicitar token y obtener artistas
    token = api.request_token(silent=True)
    artistas = api.obtener_artistas(token, id_playlist)
    lista_artistas.append(artistas)

    # Guardar los artistas como JSON
    zara.loc[idx, "artistas"] = json.dumps(artistas)

    # Guardar el progreso después de cada iteración
    zara.to_csv(output_file, index=False)

Obteniendo artistas:   0%|          | 0/873 [00:00<?, ?it/s]

267
Rate limit alcanzado. Esperando 80730 segundos...
